In [62]:
import pandas as pd
import datetime

START = '1/1/2019'
END = '5/31/2023'

In [63]:
df= pd.DataFrame(pd.date_range(START,END,freq="D"),columns=['datetime'])

df['paiva'] = df["datetime"].dt.day
df['paiva'] = df['paiva'].apply(lambda x: '{0:0>2}'.format(x))

df['kuukausi'] = df["datetime"].dt.month
df['kuukausi'] = df['kuukausi'].apply(lambda x: '{0:0>2}'.format(x))

df['kvartaali'] =df["datetime"].dt.quarter

df['vuosi'] = df["datetime"].dt.year

df['date'] = df['datetime'].dt.date

df['viikonpaiva'] = df['datetime'].dt.weekday


#df['tunti'] = df["datetime"].dt.hour
#df['tunti'] = df['tunti'].apply(lambda x: '{0:0>2}'.format(x))

df['pvm'] = df['vuosi'].astype(str) + df['kuukausi'].astype(str) + df['paiva'].astype(str) 
df = df.astype({'pvm':'int64'})

df = df.reindex(columns=['pvm','vuosi','kvartaali','kuukausi','paiva', 'date', 'viikonpaiva'])


# Adds seperate date_id column
# df = df.rename_axis('date_id').reset_index()

df.head(1)

,pvm,vuosi,kvartaali,kuukausi,paiva,date,viikonpaiva
0,20190101,2019,1,01,01,2019-01-01,1


In [64]:
def get_weekday(weekday):
    if weekday == 0:
        return 'maanantai'
    elif weekday == 1:
        return 'tiistai'
    elif weekday == 2:
        return 'keskiviikko'
    elif weekday == 3:
        return 'torstai'
    elif weekday == 4:
        return 'perjaintai'
    elif weekday == 5:
        return 'lauantai'
    else:
        return 'sunnuntai'
    
# Apply the function to the DataFrame
df['viikonpaiva'] = df['viikonpaiva'].apply(get_weekday)

df.head()

,pvm,vuosi,kvartaali,kuukausi,paiva,date,viikonpaiva
0,20190101,2019,1,01,01,2019-01-01,tiistai
1,20190102,2019,1,01,02,2019-01-02,keskiviikko
2,20190103,2019,1,01,03,2019-01-03,torstai
3,20190104,2019,1,01,04,2019-01-04,perjaintai
4,20190105,2019,1,01,05,2019-01-05,lauantai


In [44]:
#def get_season(quarter):
#    if quarter == 1:
#        return 'talvi'
#    elif quarter == 2:
#        return 'kevat'
#    elif quarter == 3:
#        return 'kesa'
#    else:
#        return 'syksy'

# Apply the function to the DataFrame
# df['kvartaali'] = df['kvartaali'].apply(get_season)

# df.head()

,pvm,vuosi,kvartaali,kuukausi,paiva,date,viikonpaiva
0,20190101,2019,talvi,01,01,2019-01-01,tiistai
1,20190102,2019,talvi,01,02,2019-01-02,keskiviikko
2,20190103,2019,talvi,01,03,2019-01-03,torstai
3,20190104,2019,talvi,01,04,2019-01-04,perjaintai
4,20190105,2019,talvi,01,05,2019-01-05,lauantai


#### Write DF to csv

In [66]:
df.to_csv('DimDate.csv', sep=',')

#### Configurations

In [67]:
import pyodbc
import pandas as pd
from warnings import simplefilter

SERVER = 'localhost'
DATABASE = 'FinalProject'
USER = 'sa'
PWD = 'Winter11'

simplefilter(action='ignore', category=UserWarning)

# Create a connection object
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)

# Create a cursor object
cursor = cnxn.cursor()

#### Create DimDate table if not exist

In [69]:
# Create a table if it doesn't exist
try:
    cursor.execute("""
    CREATE TABLE DimDate(
        aika_id int IDENTITY(1,1) Primary Key,
        pvm int,
        vuosi int,
        kvartaali int,
        kuukausi int,
        paiva int,
        date date,
        viikonpaiva varchar(15)
        )
    """)

except:
    print('Table already exists!')
    cursor.execute("TRUNCATE TABLE DimDate")

cnxn.commit()

# klo

In [49]:
# cursor.execute("DROP TABLE DimDate")

#### Load data into DimDat table from newDf (data frame)

In [70]:
df.head(1)

,pvm,vuosi,kvartaali,kuukausi,paiva,date,viikonpaiva
0,20190101,2019,1,01,01,2019-01-01,tiistai


In [71]:
for index, row in df.iterrows():
    cursor.execute("INSERT INTO DimDate values(?,?,?,?,?,?,?)",
                   row['pvm'],
                   row['vuosi'],
                   row['kvartaali'],
                   row['kuukausi'],
                   row['paiva'],
                   row['date'],
                   row['viikonpaiva']
                   )
    
cnxn.commit()


In [72]:
cursor.close()
cnxn.close()

In [114]:

# check if values in column 'B' are unique
print(df['datetime'].duplicated())

print("HELLOOO")
# show only duplicated values
print(df[df['pvm'].duplicated()])

0        False
1         True
2         True
3         True
4         True
         ...  
37196     True
37197     True
37198     True
37199     True
37200    False
Name: pvm, Length: 37201, dtype: bool
HELLOOO
            pvm  vuosi kvartaali kuukausi paiva tunti            datetime
1      20190101   2019     talvi       01    01    01 2019-01-01 01:00:00
2      20190101   2019     talvi       01    01    02 2019-01-01 02:00:00
3      20190101   2019     talvi       01    01    03 2019-01-01 03:00:00
4      20190101   2019     talvi       01    01    04 2019-01-01 04:00:00
5      20190101   2019     talvi       01    01    05 2019-01-01 05:00:00
...         ...    ...       ...      ...   ...   ...                 ...
37195  20230330   2023     talvi       03    30    19 2023-03-30 19:00:00
37196  20230330   2023     talvi       03    30    20 2023-03-30 20:00:00
37197  20230330   2023     talvi       03    30    21 2023-03-30 21:00:00
37198  20230330   2023     talvi       03    30  